In [ ]:
import configparser
import os

import numpy as np
import pandas as pd
import scipy.optimize as so

import stereotactic_electrodes as se

In [ ]:
electrodes = se.ElectrodesBrainLabACPC('stereotactic_electrodes/epi_eles.ini')

In [ ]:
DF = []

for name, _contacts in electrodes._electrodes.items():
    for n, e in _contacts._contacts.items():
        DF.append({'NAME': f'{name}_{n}',
                   'GROUP': name,
                   'N': n,
                   'X': e.x,
                   'Y': e.y,
                   'Z': e.z,
                   })

DF = pd.DataFrame(DF)

In [ ]:
DF['R'] = np.sqrt(np.square(DF.X) + np.square(DF.Y) + np.square(DF.Z))

In [ ]:
GB = DF.groupby('GROUP')

In [ ]:
DF2 = GB.apply(lambda DF: DF.iloc[np.argmax(DF.R)])

In [ ]:
DF2

In [ ]:
XYZ = DF2[['X', 'Y', 'Z']].to_numpy()

In [ ]:
def f(x):
    dr = np.sqrt(np.square(XYZ - x).sum(axis=1)) - 78.9
    return np.square(np.maximum(0, dr)).sum()

In [ ]:
minimum = so.minimize(f, [0, 0, 0])
print(minimum.fun)
print(minimum.x)

In [ ]:
for c, delta in zip('XYZ', minimum.x):
    DF[f'{c}0'] = (DF[c] - delta) / 1e3

In [ ]:
DF['R0'] = np.sqrt(np.square(DF.X0) + np.square(DF.Y0) + np.square(DF.Z0))

In [ ]:
DF.R0.max()

In [ ]:
PATH = '4SM_CSF_3_mm/uniform_coarse_3/'

In [ ]:
base_config = configparser.ConfigParser()
base_config.read(os.path.join('FEM/fem_configs/paper/', PATH, 'stereotactic.ini'))

In [ ]:
config = configparser.ConfigParser()
config.add_section('fem')

for field in ['mesh', 'config', 'element_type', 'degree']:
    config.set('fem', field, base_config.get('fem', field))
    
config.set('fem', 'solution_metadata_filename', os.path.join('solutions/paper/', PATH, 'epi_wroclaw.ini'))

In [ ]:
for _, ROW in DF.iterrows():
    config.add_section(ROW.NAME)
    config.set(ROW.NAME, 'filename', f'epi_wroclaw/{ROW.NAME}.h5')
    for c in 'XYZ':
        config.set(ROW.NAME, c.lower(), repr(ROW[f'{c}0']))

In [ ]:
config.write(open(os.path.join('FEM/fem_configs/paper/',
                               PATH,
                               'epi_wroclaw.ini'),
                  'w'))

In [ ]:
print(repr(list(DF.NAME)))

In [ ]:
PATH = '1SM/finest_3'

In [ ]:
base_config = configparser.ConfigParser()
base_config.read(os.path.join('FEM/fem_configs/paper/', PATH, 'stereotactic.ini'))

In [ ]:
config = configparser.ConfigParser()
config.add_section('fem')

for field in ['mesh', 'config', 'element_type', 'degree']:
    config.set('fem', field, base_config.get('fem', field))
    
config.set('fem', 'solution_metadata_filename', os.path.join('solutions/paper/', PATH, 'epi_wroclaw.ini'))

In [ ]:
for _, ROW in DF.iterrows():
    config.add_section(ROW.NAME)
    config.set(ROW.NAME, 'filename', f'epi_wroclaw/{ROW.NAME}.h5')
    for c in 'XYZ':
        config.set(ROW.NAME, c.lower(), repr(ROW[c]))

In [ ]:
config.write(open(os.path.join('FEM/fem_configs/paper/',
                               PATH,
                               'epi_wroclaw.ini'),
                  'w'))